In [4]:
import os
from groq import Groq
import json

# Initialize the Groq client
client = Groq()
# Specify the model to be used (we recommend Llama 3.3 70B)
MODEL = 'llama-3.1-8b-instant'

In [5]:
def calculate(expression):
    """Evaluate a mathematical expression"""
    try:
        # Attempt to evaluate the math expression
        result = eval(expression)
        return json.dumps({"result": result})
    except:
        # Return an error message if the math expression is invalid
        return json.dumps({"error": "Invalid expression"})

In [6]:
# imports calculate function from step 1
def run_conversation(user_prompt):
    # Initialize the conversation with system and user messages
    messages=[
        {
            "role": "system",
            "content": "You are a calculator assistant. Use the calculate function to perform mathematical operations and provide the results."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    # Define the available tools (i.e. functions) for our model to use
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Evaluate a mathematical expression",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "The mathematical expression to evaluate",
                        }
                    },
                    "required": ["expression"],
                },
            },
        }
    ]
    # Make the initial API call to Groq
    response = client.chat.completions.create(
        model=MODEL, # LLM to use
        messages=messages, # Conversation history
        stream=False,
        tools=tools, # Available tools (i.e. functions) for our LLM to use
        tool_choice="auto", # Let our LLM decide when to use tools
        max_completion_tokens=4096 # Maximum number of tokens to allow in our response
    )
    # Extract the response and any tool call responses
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        # Define the available tools that can be called by the LLM
        available_functions = {
            "calculate": calculate,
        }
        # Add the LLM's response to the conversation
        messages.append(response_message)

        # Process each tool call
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            # Call the tool and get the response
            function_response = function_to_call(
                expression=function_args.get("expression")
            )
            # Add the tool response to the conversation
            messages.append(
                {
                    "tool_call_id": tool_call.id, 
                    "role": "tool", # Indicates this message is from tool use
                    "name": function_name,
                    "content": function_response,
                }
            )
        # Make a second API call with the updated conversation
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        # Return the final response
        return second_response.choices[0].message.content

In [7]:
# Example usage
user_prompt = "What is 25 * 4 + 10?"
print(run_conversation(user_prompt))

The result of the expression 25 * 4 + 10 is 110.


In [8]:
# imports calculate function from step 1
user_prompt = "What is 25 * 4 + 10?"
# Initialize the conversation with system and user messages
messages=[
    {
        "role": "system",
        "content": "You are a calculator assistant. Use the calculate function to perform mathematical operations and provide the results."
    },
    {
        "role": "user",
        "content": user_prompt,
    }
]

In [10]:
# Define the available tools (i.e. functions) for our model to use
tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Evaluate a mathematical expression",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "The mathematical expression to evaluate",
                    }
                },
                "required": ["expression"],
            },
        },
    }
]

In [11]:
# Make the initial API call to Groq
response = client.chat.completions.create(
    model=MODEL, # LLM to use
    messages=messages, # Conversation history
    stream=False,
    tools=tools, # Available tools (i.e. functions) for our LLM to use
    tool_choice="auto", # Let our LLM decide when to use tools
    max_completion_tokens=4096 # Maximum number of tokens to allow in our response
)

In [16]:
response.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='02x0k9a9g', function=Function(arguments='{"expression":"25 * 4 + 10"}', name='calculate'), type='function')]

In [17]:
# Extract the response and any tool call responses
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
if tool_calls:
    # Define the available tools that can be called by the LLM
    available_functions = {
        "calculate": calculate,
    }
    # Add the LLM's response to the conversation
    messages.append(response_message)

    # Process each tool call
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        # Call the tool and get the response
        function_response = function_to_call(
            expression=function_args.get("expression")
        )
        # Add the tool response to the conversation
        messages.append(
            {
                "tool_call_id": tool_call.id, 
                "role": "tool", # Indicates this message is from tool use
                "name": function_name,
                "content": function_response,
            }
        )
    # Make a second API call with the updated conversation
    second_response = client.chat.completions.create(
        model=MODEL,
        messages=messages
    )

In [18]:
# Return the final response
second_response.choices[0].message.content

'The result of the operation 25 * 4 + 10 is 110.'

In [26]:
second_response.choices[0].message.content

'The result of the operation 25 * 4 + 10 is 110.'

In [27]:
# Define weather tools
def get_temperature(location: str):
    # This is a mock tool/function. In a real scenario, you would call a weather API.
    temperatures = {"New York": "22°C", "London": "18°C", "Tokyo": "26°C", "Sydney": "20°C"}
    return temperatures.get(location, "Temperature data not available")

In [28]:
def get_weather_condition(location: str):
    # This is a mock tool/function. In a real scenario, you would call a weather API.
    conditions = {"New York": "Sunny", "London": "Rainy", "Tokyo": "Cloudy", "Sydney": "Clear"}
    return conditions.get(location, "Weather condition data not available")

In [29]:
# Define system messages and tools
messages = [
    {"role": "system", "content": "You are a helpful weather assistant."},
    {"role": "user", "content": "What's the weather and temperature like in New York and London? Respond with one sentence for each city. Use tools to get the information."},
]

In [30]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_temperature",
            "description": "Get the temperature for a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The name of the city",
                    }
                },
                "required": ["location"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_weather_condition",
            "description": "Get the weather condition for a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The name of the city",
                    }
                },
                "required": ["location"],
            },
        },
    }
]

In [33]:
# Make the initial request
response = client.chat.completions.create(
    model=model, messages=messages, tools=tools, tool_choice="auto", max_completion_tokens=4096, temperature=0.5
)

In [34]:
response_message = response.choices[0].message
tool_calls = response_message.tool_calls

In [36]:
tool_calls

[ChatCompletionMessageToolCall(id='jjdv6j976', function=Function(arguments='{"location":"New York"}', name='get_temperature'), type='function'),
 ChatCompletionMessageToolCall(id='ptvxjswry', function=Function(arguments='{"location":"London"}', name='get_temperature'), type='function'),
 ChatCompletionMessageToolCall(id='9z3hzey2p', function=Function(arguments='{"location":"New York"}', name='get_weather_condition'), type='function'),
 ChatCompletionMessageToolCall(id='c2cy59955', function=Function(arguments='{"location":"London"}', name='get_weather_condition'), type='function')]

In [37]:
# Process tool calls
messages.append(response_message)

In [38]:
messages

[{'role': 'system', 'content': 'You are a helpful weather assistant.'},
 {'role': 'user',
  'content': "What's the weather and temperature like in New York and London? Respond with one sentence for each city. Use tools to get the information."},
 ChatCompletionMessage(content=None, role='assistant', executed_tools=None, function_call=None, reasoning=None, tool_calls=[ChatCompletionMessageToolCall(id='jjdv6j976', function=Function(arguments='{"location":"New York"}', name='get_temperature'), type='function'), ChatCompletionMessageToolCall(id='ptvxjswry', function=Function(arguments='{"location":"London"}', name='get_temperature'), type='function'), ChatCompletionMessageToolCall(id='9z3hzey2p', function=Function(arguments='{"location":"New York"}', name='get_weather_condition'), type='function'), ChatCompletionMessageToolCall(id='c2cy59955', function=Function(arguments='{"location":"London"}', name='get_weather_condition'), type='function')])]

In [39]:
available_functions = {
    "get_temperature": get_temperature,
    "get_weather_condition": get_weather_condition,
}

In [48]:
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(**function_args)

    messages.append(
        {
            "role": "tool",
            "content": str(function_response),
            "tool_call_id": tool_call.id,
        }
    )

In [49]:
# Make the final request with tool call results
final_response = client.chat.completions.create(
    model=model, messages=messages, tools=tools, tool_choice="auto", max_completion_tokens=4096
)

In [50]:
print(final_response.choices[0].message.content)

Note that the actual temperatures and weather conditions may change frequently. The above responses are just examples. If you want up-to-date information, you can use APIs or other tools that provide real-time weather data. 

For this example, I assumed the temperature in New York is 22°C and in London is 18°C, and the weather condition in New York is Sunny and in London is Rainy, which might not reflect the actual current weather in these cities.


In [52]:
import instructor
from pydantic import BaseModel, Field

In [53]:
# Define the tool schema
tool_schema = {
    "name": "get_weather_info",
    "description": "Get the weather information for any location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The location for which we want to get the weather information (e.g., New York)"
            }
        },
        "required": ["location"]
    }
}

In [54]:
# Define the Pydantic model for the tool call
class ToolCall(BaseModel):
    input_text: str = Field(description="The user's input text")
    tool_name: str = Field(description="The name of the tool to call")
    tool_parameters: str = Field(description="JSON string of tool parameters")

In [55]:
class ResponseModel(BaseModel):
    tool_calls: list[ToolCall]

In [57]:
# Patch Groq() with instructor
client = instructor.from_groq(Groq(), mode=instructor.Mode.JSON)

In [58]:
def run_conversation(user_prompt):
    # Prepare the messages
    messages = [
        {
            "role": "system",
            "content": f"You are an assistant that can use tools. You have access to the following tool: {tool_schema}"
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]

    # Make the Groq API call
    response = client.chat.completions.create(
        model=model,
        response_model=ResponseModel,
        messages=messages,
        temperature=0.5,
        max_completion_tokens=1000,
    )

    return response.tool_calls

In [59]:
# Example usage
user_prompt = "What's the weather like in San Francisco?"
tool_calls = run_conversation(user_prompt)

In [60]:
tool_calls

[ToolCall(input_text="What's the weather like in San Francisco?", tool_name='get_weather_info', tool_parameters='{"location": "San Francisco"}')]

In [61]:
for call in tool_calls:
    print(f"Input: {call.input_text}")
    print(f"Tool: {call.tool_name}")
    print(f"Parameters: {call.tool_parameters}")
    print()

Input: What's the weather like in San Francisco?
Tool: get_weather_info
Parameters: {"location": "San Francisco"}



In [62]:
# Define the tool schema
tool_schema = {
    "name": "get_weather_info",
    "description": "Get the weather information for any location.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The location for which we want to get the weather information (e.g., New York)"
            }
        },
        "required": ["location"]
    }
}

In [63]:
# Define the Pydantic model for the tool call
class ToolCall(BaseModel):
    input_text: str = Field(description="The user's input text")
    tool_name: str = Field(description="The name of the tool to call")
    tool_parameters: str = Field(description="JSON string of tool parameters")

In [64]:
class ResponseModel(BaseModel):
    tool_calls: list[ToolCall]

In [65]:
# Patch Groq() with instructor
client = instructor.from_groq(Groq(), mode=instructor.Mode.JSON)

In [66]:
def run_conversation(user_prompt):
    # Prepare the messages
    messages = [
        {
            "role": "system",
            "content": f"You are an assistant that can use tools. You have access to the following tool: {tool_schema}"
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]

    # Make the Groq API call
    response = client.chat.completions.create(
        model=model,
        response_model=ResponseModel,
        messages=messages,
        temperature=0.5,
        max_completion_tokens=1000,
    )

    return response.tool_calls

In [67]:
# Example usage
user_prompt = "What's the weather like in San Francisco?"
tool_calls = run_conversation(user_prompt)

In [68]:
for call in tool_calls:
    print(f"Input: {call.input_text}")
    print(f"Tool: {call.tool_name}")
    print(f"Parameters: {call.tool_parameters}")
    print()

Input: 
Tool: get_weather_info
Parameters: {"location": "San Francisco"}

